# Extreme Fill 2D Example

In [23]:
%reload_ext yamlmagic

from extremefill2D.fextreme import init_sim, restart_sim

In [2]:
treant = init_sim('scripts/params.json', 'data', tags=['debug'])

In [3]:
treant = restart_sim(treant, 10)

In [4]:
import xarray
xarray.open_dataset(sorted(treant.glob('data*.nc'))[-1].abspath)

<xarray.Dataset>
Dimensions:             (cupric: 5510, distance: 5510, dx: 38, dy: 145, potential: 5510, suppressor: 5510, theta: 5510, x: 5510, y: 5510)
Coordinates:
  * potential           (potential) float64 0.25 0.25 0.25 0.25 0.25 0.25 ...
  * x                   (x) float64 4.356e-07 1.146e-06 1.718e-06 2.302e-06 ...
  * distance            (distance) float64 -5.406e-06 -5.087e-06 -4.802e-06 ...
  * cupric              (cupric) float64 1e+03 1e+03 1e+03 1e+03 1e+03 1e+03 ...
  * theta               (theta) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
  * dy                  (dy) float64 1.321e-06 7.454e-07 6.776e-07 6.16e-07 ...
  * dx                  (dx) float64 8.6e-07 5.6e-07 5.84e-07 5.84e-07 ...
  * y                   (y) float64 -6.094e-05 -6.094e-05 -6.094e-05 ...
  * suppressor          (suppressor) float64 0.02 0.02 0.02 0.02 0.02 0.02 ...
Data variables:
    elapsed_time        float64 0.1753
    time_step_duration  float64 0.02594

In [5]:
data = xarray.open_dataset(sorted(treant.glob('data*.nc'))[-1].abspath)

In [6]:
min(data.x.values)
max(data.y.values)
min(data.y.values)

-6.0939480000000014e-05

In [8]:
x_data = data.x.values
y_data = data.y.values
values = data.distance.values

In [13]:
import numpy as np

from scipy.interpolate import griddata

dx = 2e-7
xi = np.linspace(min(x_data), max(x_data), (max(x_data) - min(x_data)) // dx)
yi = np.linspace(min(y_data), max(y_data), (max(y_data) - min(y_data)) // dx)
# grid the data.
zi = griddata((x_data, y_data), values, (xi[None,:], yi[:,None]), method='cubic')
print(zi.shape)

(995, 228)


In [19]:
from skimage import measure

contours = measure.find_contours(zi, 0.0)

print(len(contours))
print(contours)

1
[array([[ 304.38321084,    0.        ],
       [ 304.38418157,    1.        ],
       [ 304.38415359,    2.        ],
       ..., 
       [ 304.38229422,  225.        ],
       [ 304.38228902,  226.        ],
       [ 304.38228045,  227.        ]])]


In [26]:
%%yaml vega_yaml
---
width: 500
height: 500

data:
  - name: contour_data
    values:
      - x: 0.0
        y: 0.0
      - x: 1.0
        y: 0.5
      - x: 2.0
        y: 2.0

axes:
  - type: x
    scale: x
  - type: y
    scale: y

scales:
  - name: x
    nice: true
    range: width
    domain:
      data: contour_data
      field: x
  - name: y
    nice: true
    range: height
    domain:
      data: contour_data
      field: y

marks:
  - type: line
    from:
      data: contour_data
    properties:
      enter:
        x:
          scale: x
          field: x
        y:
          scale: y
          field: y

<IPython.core.display.Javascript object>

In [27]:
print(vega_yaml)

{'height': 500, 'width': 500, 'scales': [{'name': 'x', 'domain': {'field': 'x', 'data': 'contour_data'}, 'nice': True, 'range': 'width'}, {'name': 'y', 'domain': {'field': 'y', 'data': 'contour_data'}, 'nice': True, 'range': 'height'}], 'data': [{'name': 'contour_data', 'values': [{'x': 0.0, 'y': 0.0}, {'x': 1.0, 'y': 0.5}, {'x': 2.0, 'y': 2.0}]}], 'marks': [{'from': {'data': 'contour_data'}, 'properties': {'enter': {'x': {'field': 'x', 'scale': 'x'}, 'y': {'field': 'y', 'scale': 'y'}}}, 'type': 'line'}], 'axes': [{'scale': 'x', 'type': 'x'}, {'scale': 'y', 'type': 'y'}]}
